In [39]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import itertools as iter
from random import shuffle

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)


Num GPUs Available:  1
/mnt/e/ML/MonetGAN


In [2]:
T=300
beta = np.linspace(1e-4, 0.04, T)
alpha = 1-beta
alphaBar = [np.prod(alpha[0:i+1]) for i,el in enumerate(alpha)]
IMAGE_SIZE = [256,256]
tValues = np.arange(0,T)
# alphaBar = np.cos((tValues/(T+1) + 0.008)/(1 + 0.008) * np.pi/2) 
# np.random.shuffle(tValues)




def forwardStep(x0Image,t):
    noise = np.random.normal(0, 1, x0Image.shape)
    noisy = np.sqrt(alphaBar[t])*x0Image + np.sqrt(1-alphaBar[t])*noise
    return noise, noisy


In [3]:


def backwardStep(xt, t, predNoise=None, x0Pred=None):

    if x0Pred is None:
        x0Pred=(xt - np.sqrt(1-alphaBar[t])*predNoise)/np.sqrt(alphaBar[t])

    if t==0:
        sample = x0Pred
        noise= np.zeros(xt.shape)
    else:
        # estimate mean
        meanPred= x0Pred * (np.sqrt(alphaBar[t])*beta[t])/((1-alphaBar[t])*np.sqrt(alpha[t])) + xt*(alpha[t]-alphaBar[t])/((1-alphaBar[t])*np.sqrt(alpha[t]))

        # compute variance
        betaPred = np.sqrt(beta[t])

        sample = meanPred + betaPred*np.random.normal(0,1,xt.shape)

        noise=(sample-x0Pred*np.sqrt(alphaBar[t-1]))/np.sqrt(1-alphaBar[t-1])

    return sample, noise



In [4]:
def postprocessImage(im):
    return np.clip(im/2+0.5,0,1)

## Tensorflow

In [5]:
model = tf.keras.models.load_model(os.path.join(DATA_PATH, "diffusionModel_epoch_2_step_800"))
IMAGE_SIZE=[256,256]

2023-08-25 21:36:28.585944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-25 21:36:28.586295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-25 21:36:28.586596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-25 21:36:29.058280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-25 21:36:29.058685: I tensorflow/compile

In [6]:
def generateImages(BatchSize=64):
    initInput = tf.random.normal((BatchSize, *IMAGE_SIZE, 3), dtype=tf.float32)
    samples = initInput

    for t in np.arange(0,T)[::-1]:
        predX0 = model((samples, tf.repeat(t, repeats=BatchSize)), training=False)
        xt = samples
        
        new, noise = backwardStep(xt, t, x0Pred=predX0)

        samples = new
        
    return samples



In [40]:
N=70

for n in tqdm(range(N)):
    images = generateImages(100)
    images = postprocessImage(images)
    for i in range(images.shape[0]):
        im = Image.fromarray(np.array(images[i,:,:,:]*255).astype(np.uint8), "RGB" )
        im.save(os.path.join(DATA_PATH,"results/"+ str(n*100+i)+".jpg"),quality=80)
        

  0%|          | 0/70 [00:00<?, ?it/s]

In [29]:
im = Image.fromarray(np.array(images[0,:,:,:]*255).astype(np.uint8), "RGB" )
# np.array(img).astype(np.uint8).transpose(1,2,0)

In [27]:
os.path.join(DATA_PATH, "results", str(i)+".jpg")

'/mnt/e/ML/MonetGAN/results/0.jpg'

In [36]:
im.save(os.path.join(DATA_PATH,"results/"+ str(i)+".jpg"))